In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import xarray as xr
import numpy as np
import toml
import particles_vis_tooling as pv
from env import PARTICLE_SIM_FOLDER, CONFIG_FOLDER, PARTICLE_PLOT_FOLDER, beaching_strats, coast_shapes, resolutions
import os

In [ ]:
toml_fnames = [
    "flat_1km_oninkb.toml",
    "concave_1km_oninkb.toml",
    "convex_1km_oninkb.toml",
]


simulations = {fname: pv.ParticlesSimulation.from_toml(fname, all_particles=True) for fname in toml_fnames}

In [ ]:
import math
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib
from tqdm import tqdm

from env import PARTICLE_PLOT_FOLDER

def rand():
    return np.random.random(10)


class UpdateDist:
    def __init__(self, ax, sim_obj):
        self.sim_obj = sim_obj
        self.ds_trajectories = sim_obj.ds_trajectories
        self.ds_fieldset = sim_obj.ds_fieldset
        
        self.field = self.ds_fieldset.land.plot(ax=ax, cmap = "Greys")
        # Remove colourbar
        ax.collections[0].colorbar.remove()

        self.scat = ax.scatter([], [], color = 'red', s=1)
        self.ax = ax

        # Set up plot parameters


    def __call__(self, i):
        # This way the plot can continuously run and we just keep
        # watching new realizations of the process
        ds = self.ds_trajectories.isel(obs = i)
        self.scat.set_offsets(np.vstack([ds.lon.values, ds.lat.values]).T)

        self.ax.set(
            title = f"{self.sim_obj.name}, days={ds.days_since_start.item():.2f}"
        )
        return self.scat,

ANIMATION = True
with plt.rc_context(matplotlib.rcParamsDefault):
    if ANIMATION:
        pbar = tqdm(simulations.values())
        for sim_obj in pbar:
            animation_path = PARTICLE_PLOT_FOLDER / "animations" / f"{sim_obj.name}.mp4"
            pbar.set_description(sim_obj.name)
            if animation_path.exists():
                continue
            fig, ax = plt.subplots()
            ud = UpdateDist(ax, sim_obj)
            anim = FuncAnimation(fig, ud, frames=int((24 / 0.5) * 15), interval=100, blit=True)

            writervideo = FFMpegWriter(fps=60) 
            anim.save(animation_path, writer=writervideo)